In [0]:
%pip install fast_ml
%pip install tqdm
%pip install prophet

Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import pandas as pd
import tqdm
import numpy as np
import matplotlib.pyplot as plt  
from datetime import datetime
import statistics
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.ensemble import IsolationForest

/databricks/python/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [0]:

df = pd.read_csv("/dbfs/FileStore/shared_uploads/chien.liu@hellofresh.com/within_covid.csv", parse_dates=['ds'])
df_covid = pd.read_csv("/dbfs/FileStore/shared_uploads/chien.liu@hellofresh.com/Covid.csv")

In [0]:
df_2 = df[df['ds'] <= '2022-08-31']
df_2.tail()

In [0]:
activations = df_2[['ds', 'y']]
activations = activations.rename(columns={'simulated':'y'})
activations['ds'] = pd.to_datetime(activations['ds'])
activations_index = activations.set_index(['ds'])
activations_index.head()

In [0]:
activations_index = activations_index.resample('W').sum().ffill()
activations_index.head()

In [0]:
within_covid = activations_index.reset_index().copy()
print(len(within_covid))

In [0]:
within_covid['ds'] = pd.date_range(start='2018-01-07', periods=243, freq='W') ##config the time index to fit in prophet ds

In [0]:
f, ax = plt.subplots(figsize=(30, 5))

actual_index = within_covid.set_index(['ds'])
#forecast_index = forecast.set_index(['ds'])

actual_index['y'].plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)
#forecast_index['yhat'].plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
plt.title('Actual Activations')
plt.legend()
plt.show()

#Baseline Model

## Without Covid Model

In [0]:
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot, plot_cross_validation_metric
from prophet.diagnostics import cross_validation, performance_metrics

In [0]:
m = Prophet(
  # yearly_seasonality= False, 
  #                       weekly_seasonality= 'auto',
  #                       changepoint_prior_scale= 0.05,
  #                       seasonality_prior_scale=10,
  #                       holidays_prior_scale=0.1,
  #                       seasonality_mode = 'multiplicative'
                        )

m.fit(within_covid)

future = m.make_future_dataframe(17, freq='W')
forecast = m.predict(future)

fig = m.plot_components(forecast)

In [0]:
m.plot(forecast, uncertainty=True)
plt.show()

In [0]:
# Initial training period.
initial= 730
initial= str(initial)+' days'

#Period length that we perform the cross validation for.
period= 365
period=str(period)+' days'

#Horizon of prediction essentially for each fold.
horizon = 120
horizon=str(horizon)+' days'
fb_cv=cross_validation(m,initial=initial,period=period,
horizon=horizon)

# Performance Metrics of fb_cv
cv = performance_metrics(fb_cv)
cv

In [0]:
cv.mean()

In [0]:
from sklearn.metrics import mean_absolute_percentage_error,  r2_score, mean_squared_error

y_true = activations_index['y']
y_pred = forecast['yhat'].iloc[:243]

print('mape:', mean_absolute_percentage_error(y_true, y_pred))
print('r2:', r2_score(y_true, y_pred))
print('RMSE', mean_squared_error(y_true, y_pred, squared=False))

##Adding Seasonality Mode, Seasonal Parameters, and Holidays

In [0]:
import itertools
import numpy as np
import pandas as pd

param_grid = {  
    #'changepoint_prior_scale': [0.001, 0.01, 0.05, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 5.0, 10.0],
    'holidays_prior_scale': [0.1, 1.0, 5.0, 10],
    'seasonality_mode': ['additive', 'multiplicative']
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
mape = []  # Store the RMSEs for each params here

# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params)
    m.add_country_holidays(country_name='DE')
    m.fit(within_covid)  # Fit model with given params

    df_cv = cross_validation(m, initial='730 days', period='120 days', horizon='90 days', parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    mape.append(df_p['mape'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['mape'] = mape
print(tuning_results)

14:48:58 - cmdstanpy - INFO - Chain [1] start processing
14:48:58 - cmdstanpy - INFO - Chain [1] done processing
14:48:59 - cmdstanpy - INFO - Chain [1] start processing
14:48:59 - cmdstanpy - INFO - Chain [1] done processing
14:49:01 - cmdstanpy - INFO - Chain [1] start processing
14:49:01 - cmdstanpy - INFO - Chain [1] done processing
14:49:02 - cmdstanpy - INFO - Chain [1] start processing
14:49:02 - cmdstanpy - INFO - Chain [1] done processing
14:49:03 - cmdstanpy - INFO - Chain [1] start processing
14:49:03 - cmdstanpy - INFO - Chain [1] done processing
14:49:05 - cmdstanpy - INFO - Chain [1] start processing
14:49:05 - cmdstanpy - INFO - Chain [1] done processing
14:49:06 - cmdstanpy - INFO - Chain [1] start processing
14:49:06 - cmdstanpy - INFO - Chain [1] done processing
14:49:07 - cmdstanpy - INFO - Chain [1] start processing
14:49:07 - cmdstanpy - INFO - Chain [1] done processing
14:49:08 - cmdstanpy - INFO - Chain [1] start processing
14:49:08 - cmdstanpy - INFO - Chain [1] done processing
14:49:10 - cmdstanpy - INFO - Chain [1] start processing
14:49:10 - cmdstanpy - INFO - Chain [1] done processing
14:49:11 - cmdstanpy - INFO - Chain [1] start processing
14:49:11 - cmdstanpy - INFO - Chain [1] done processing
14:49:12 - cmdstanpy - INFO - Chain [1] start processing
14:49:12 - cmdstanpy - INFO - Chain [1] done processing
14:49:14 - cmdstanpy - INFO - Chain [1] start processing
14:49:14 - cmdstanpy - INFO - Chain [1] done processing
14:49:15 - cmdstanpy - INFO - Chain [1] start processing
14:49:15 - cmdstanpy - INFO - Chain [1] done processing
14:49:16 - cmdstanpy - INFO - Chain [1] start processing
14:49:16 - cmdstanpy - INFO - Chain [1] done processing
14:49:17 - cmdstanpy - INFO - Chain [1] start processing
14:49:17 - cmdstanpy - INFO - Chain [1] done processing
14:49:19 - cmdstanpy - INFO - Chain [1] start processing
14:49:19 - cmdstanpy - INFO - Chain [1] done processing
14:49:20 - cmdstanpy - INFO - Chain [1] start processing
14:49:20 - cmdstanpy - INFO - Chain [1] done processing
14:49:21 - cmdstanpy - INFO - Chain [1] start processing
14:49:21 - cmdstanpy - INFO - Chain [1] done processing
14:49:23 - cmdstanpy - INFO - Chain [1] start processing
14:49:23 - cmdstanpy - INFO - Chain [1] done processing
14:49:24 - cmdstanpy - INFO - Chain [1] start processing
14:49:24 - cmdstanpy - INFO - Chain [1] done processing
14:49:25 - cmdstanpy - INFO - Chain [1] start processing
14:49:25 - cmdstanpy - INFO - Chain [1] done processing
14:49:27 - cmdstanpy - INFO - Chain [1] start processing
14:49:27 - cmdstanpy - INFO - Chain [1] done processing
14:49:28 - cmdstanpy - INFO - Chain [1] start processing
14:49:28 - cmdstanpy - INFO - Chain [1] done processing
14:49:29 - cmdstanpy - INFO - Chain [1] start processing
14:49:29 - cmdstanpy - INFO - Chain [1] done processing
14:49:31 - cmdstanpy - INFO - Chain [1] start processing
14:49:31 - cmdstanpy - INFO - Chain [1] done processing
14:49:32 - cmdstanpy - INFO - Chain [1] start processing
14:49:32 - cmdstanpy - INFO - Chain [1] done processing
14:49:33 - cmdstanpy - INFO - Chain [1] start processing
14:49:33 - cmdstanpy - INFO - Chain [1] done processing
14:49:34 - cmdstanpy - INFO - Chain [1] start processing
14:49:35 - cmdstanpy - INFO - Chain [1] done processing
14:49:36 - cmdstanpy - INFO - Chain [1] start processing
14:49:36 - cmdstanpy - INFO - Chain [1] done processing

In [0]:
best_params = all_params[np.argmin(mape)]
print(best_params)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1450724931318692> in <module> 
 ----> 1 best_params = all_params [ np . argmin ( mape ) ] 
 2 print ( best_params ) 

 NameError : name 'all_params' is not defined

In [0]:
m1 = Prophet(
            yearly_seasonality= 'auto', 
            weekly_seasonality= 'auto',
            # changepoint_prior_scale= 0.01,
            seasonality_prior_scale=10.0,
            holidays_prior_scale=0.1,
            seasonality_mode = 'multiplicative'
                        )
m1.add_country_holidays(country_name='DE')
m1.fit(within_covid)

future1 = m1.make_future_dataframe(17, freq='W')
forecast1 = m1.predict(future1)

fig = m1.plot_components(forecast1)

In [0]:
m1.plot(forecast1, uncertainty=True)
plt.show()

In [0]:
# Initial training period.
initial= 730
initial= str(initial)+' days'

#Period length that we perform the cross validation for.
period= 365
period=str(period)+' days'

#Horizon of prediction essentially for each fold.
horizon = 120
horizon=str(horizon)+' days'
fb_cv=cross_validation(m1,initial=initial,period=period,
horizon=horizon)

# Performance Metrics of fb_cv
cv = performance_metrics(fb_cv)
cv.head()
cv.mean()

In [0]:
from sklearn.metrics import mean_absolute_percentage_error,  r2_score, mean_squared_error

y_true = activations_index['y']
y_pred = forecast1['yhat'].iloc[:243]

print('mape:', mean_absolute_percentage_error(y_true, y_pred))
print('r2:', r2_score(y_true, y_pred))
print('RMSE', mean_squared_error(y_true, y_pred, squared=False))

## Adding Holidays

In [0]:
real_holidays = pd.DataFrame({
  'holiday': 'real_holidays',
  'ds': pd.to_datetime(['2018-04-01', '2018-12-25', '2019-04-21', '2019-12-25', '2020-04-12', '2020-12-25', '2021-04-04', '2021-12-25', '2022-04-14', '2022-12-25']),
  'lower_window': -3,
  'upper_window': 3,
})
vacations = pd.DataFrame({
  'holiday': 'vacations',
  'ds': pd.to_datetime(['2018-07-15', '2018-08-15', '2019-07-15', '2019-08-15', '2020-07-15', '2020-08-15', '2021-07-15', '2021-08-15', '2022-07-15', '2022-08-15']),
  'lower_window': -30,
  'upper_window': 30,
})
compaigns = pd.DataFrame({
  'holiday': 'compaigns',
  'ds': pd.to_datetime(['2018-03-01', '2018-09-01', '2019-03-01', '2019-09-01', '2020-03-01', '2020-09-01', '2021-03-01', '2021-09-01', '2022-03-01', '2022-09-01']),
  'lower_window': 0,
  'upper_window': 15,
})
holidays = pd.concat((real_holidays, vacations, compaigns))

In [0]:
def is_nfl_season(ds):
    date = pd.to_datetime(ds)
    return (date.month >= 9 or date.month <= 6)

within_covid['on_season'] = within_covid['ds'].apply(is_nfl_season)
within_covid['off_season'] = ~within_covid['ds'].apply(is_nfl_season)

In [0]:
m2 = Prophet(
            yearly_seasonality= False, 
            weekly_seasonality= 'auto',
            #changepoint_prior_scale= 0.05,
            seasonality_prior_scale=10,
            holidays_prior_scale=0.1,
            holidays= holidays,
            seasonality_mode = 'multiplicative'
                        )
m2.add_country_holidays(country_name='DE')
# m2.add_seasonality('quarterly', period=91.25, fourier_order=5, mode='additive')
# m2.add_seasonality('weekly', period=7, fourier_order=10, mode='multiplicative')

m2.add_seasonality(name='yearly_on_season', period=365, fourier_order=10, condition_name='on_season', mode='multiplicative')
m2.add_seasonality(name='yearly_off_season', period=365, fourier_order=10, condition_name='off_season', mode='multiplicative')

m2.fit(within_covid)

future2 = m2.make_future_dataframe(17, freq='W')
future2['on_season'] = future2['ds'].apply(is_nfl_season)
future2['off_season'] = ~future2['ds'].apply(is_nfl_season)

forecast2 = m2.predict(future2)

fig = m2.plot_components(forecast2)

In [0]:
m2.plot(forecast2, uncertainty=True)
plt.show()

In [0]:
# Initial training period.
initial= 730
initial= str(initial)+' days'

#Period length that we perform the cross validation for.
period= 365
period=str(period)+' days'

#Horizon of prediction essentially for each fold.
horizon = 120
horizon=str(horizon)+' days'
fb_cv=cross_validation(m2,initial=initial,period=period,
horizon=horizon)

# Performance Metrics of fb_cv
cv = performance_metrics(fb_cv)

cv.mean()

In [0]:
forecast2[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [0]:
f, ax = plt.subplots(figsize=(30, 5))

actual_index = within_covid.set_index(['ds'])
forecast_index = forecast2.set_index(['ds'])

actual_index['y'].plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)
forecast_index['yhat'].plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
plt.title('Actual v.s. Forecast')
plt.legend()
plt.show()

In [0]:
from sklearn.metrics import mean_absolute_percentage_error,  r2_score, mean_squared_error

y_true = activations_index['y']
y_pred = forecast2['yhat'].iloc[:243]

print('mape:', mean_absolute_percentage_error(y_true, y_pred))
print('r2:', r2_score(y_true, y_pred))
print('RMSE', mean_squared_error(y_true, y_pred, squared=False))

##Adding More Seasonality

In [0]:
m3 = Prophet(
            yearly_seasonality= False, 
            weekly_seasonality= 'auto',
            #changepoint_prior_scale= 0.05,
            seasonality_prior_scale=10,
            holidays_prior_scale=0.1,
            holidays= holidays,
            seasonality_mode = 'multiplicative'
                        )
m3.add_country_holidays(country_name='DE')

m3.add_seasonality('quarterly', period=91.25, fourier_order=5, mode='multiplicative')
#m3.add_seasonality('weekly', period=7, fourier_order=10, mode='multiplicative')

m3.add_seasonality(name='yearly_on_season', period=365, fourier_order=10, condition_name='on_season', mode='multiplicative')
m3.add_seasonality(name='yearly_off_season', period=365, fourier_order=10, condition_name='off_season', mode='multiplicative')

m3.fit(within_covid)

future3 = m3.make_future_dataframe(17, freq='W')
future3['on_season'] = future3['ds'].apply(is_nfl_season)
future3['off_season'] = ~future3['ds'].apply(is_nfl_season)

forecast3 = m3.predict(future3)

fig = m3.plot_components(forecast3)

In [0]:
m3.plot(forecast3, uncertainty=True)
plt.show()

In [0]:
# Initial training period.
initial= 730
initial= str(initial)+' days'

#Period length that we perform the cross validation for.
period= 365
period=str(period)+' days'

#Horizon of prediction essentially for each fold.
horizon = 120
horizon=str(horizon)+' days'
fb_cv=cross_validation(m3,initial=initial,period=period,
horizon=horizon)

# Performance Metrics of fb_cv
cv = performance_metrics(fb_cv)

cv.mean()

In [0]:
from sklearn.metrics import mean_absolute_percentage_error,  r2_score, mean_squared_error

y_true = activations_index['y']
y_pred = forecast3['yhat'].iloc[:243]

print('mape:', mean_absolute_percentage_error(y_true, y_pred))
print('r2:', r2_score(y_true, y_pred))
print('RMSE', mean_squared_error(y_true, y_pred, squared=False))

In [0]:
f, ax = plt.subplots(figsize=(30, 5))

actual_index = within_covid.set_index(['ds'])
forecast_index = forecast3.set_index(['ds'])

actual_index['y'].plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)
forecast_index['yhat'].plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
plt.title('Actual v.s. Forecast')
plt.legend()
plt.show()

# Adding Covid

In [0]:
covid = df_covid[df_covid['countriesAndTerritories'] == 'Germany'].sort_values(by=['year', 'month', 'day'])

In [0]:
covid = covid.iloc[24:]
covid = covid.rename(columns={'dateRep':'ds'})
covid['ds'] = pd.date_range(start='2020-03-01', periods=len(covid), freq='D') ##config the time index to fit in prophet ds
covid['ds'] = pd.to_datetime(covid['ds'])
covid.head(5)

In [0]:
covid_index = covid.set_index(['ds']).resample('W').sum().ffill()
covid_index = covid_index.iloc[:-3]
covid_index = covid_index[['cases', 'deaths']].reset_index()
covid_index.tail()

In [0]:
within_covid_activation = within_covid.merge(covid_index, how='left', on='ds')
within_covid_activation = within_covid_activation.fillna(0)
within_covid_activation.head()

In [0]:
date_df = pd.date_range(start = '2018-01-07', end = '2022-12-31', freq = 'W')
date_df = pd.DataFrame(date_df)
#date_df = date_df.reset_index()
date_df = date_df.rename(columns={0:'ds'})
within_covid_activation = within_covid_activation.merge(date_df, on = 'ds', how = 'right')
within_covid_activation['cases'] = within_covid_activation['cases'].fillna(0)
within_covid_activation['deaths'] = within_covid_activation['deaths'].fillna(0)
within_covid_activation

In [0]:
real_holidays = pd.DataFrame({
  'holiday': 'real_holidays',
  'ds': pd.to_datetime(['2018-04-01', '2018-12-25', '2019-04-21', '2019-12-25', '2020-04-12', '2020-12-25', '2021-04-04', '2021-12-25', '2022-04-14', '2022-12-25']),
  'lower_window': -3,
  'upper_window': 3,
})
vacations = pd.DataFrame({
  'holiday': 'vacations',
  'ds': pd.to_datetime(['2018-07-05', '2018-08-05', '2019-07-05', '2019-08-05', '2020-07-05', '2020-08-05', '2021-07-05', '2021-08-05', '2022-07-05', '2022-08-05']),
  'lower_window': -15,
  'upper_window': 15,
})
compaigns = pd.DataFrame({
  'holiday': 'compaigns',
  'ds': pd.to_datetime(['2018-03-01', '2018-09-01', '2019-03-01', '2019-09-01', '2020-03-01', '2020-09-01', '2021-03-01', '2021-09-01', '2022-03-01', '2022-09-01']),
  'lower_window': 0,
  'upper_window': 15,
})
lockdowns = pd.DataFrame([
    {'holiday': 'lockdown_1', 'ds': '2020-03-21', 'lower_window': 0, 'ds_upper': '2020-06-06'},
    {'holiday': 'lockdown_2', 'ds': '2020-11-14', 'lower_window': 0, 'ds_upper': '2021-04-27'},
    {'holiday': 'lockdown_3', 'ds': '2021-10-13', 'lower_window': 0, 'ds_upper': '2022-01-31'},
])
for t_col in ['ds', 'ds_upper']:
    lockdowns[t_col] = pd.to_datetime(lockdowns[t_col])
lockdowns['upper_window'] = (lockdowns['ds_upper'] - lockdowns['ds']).dt.days

holidays = pd.concat((real_holidays, vacations, compaigns, lockdowns))

In [0]:
def is_nfl_season(ds):
    date = pd.to_datetime(ds)
    return (date.month >= 9 or date.month <= 6)

within_covid_activation['on_season'] = within_covid_activation['ds'].apply(is_nfl_season)
within_covid_activation['off_season'] = ~within_covid_activation['ds'].apply(is_nfl_season)

In [0]:
within_covid_activation

In [0]:
m4 = Prophet(
            yearly_seasonality= False, 
            weekly_seasonality= True,
            changepoint_prior_scale= 0.5,
            seasonality_prior_scale=10,
            holidays_prior_scale=0.5,
            holidays= holidays,
            seasonality_mode = 'multiplicative'
                        )
m4.add_country_holidays(country_name='DE')
m4.add_seasonality('quarterly', period=91.25, fourier_order=5, mode='multiplicative')
#m4.add_seasonality('weekly', period=7, fourier_order=10, mode='multiplicative')

m4.add_seasonality(name='yearly_on_season', period=365, fourier_order=10, condition_name='on_season', mode='multiplicative')
m4.add_seasonality(name='yearly_off_season', period=365, fourier_order=10, condition_name='off_season', mode='multiplicative')

m4.add_regressor('cases', standardize='auto', mode='multiplicative')
#m4.add_regressor('deaths', standardize='auto', mode='multiplicative')

m4.fit(within_covid_activation)

# future4 = m4.make_future_dataframe(17, freq='W')
# future4['on_season'] = future4['ds'].apply(is_nfl_season)
# future4['off_season'] = ~future4['ds'].apply(is_nfl_season)

forecast4 = m4.predict(within_covid_activation)

fig = m4.plot_components(forecast4)

In [0]:
m4.plot(forecast4, uncertainty=True)
plt.show()

In [0]:
# Initial training period.
initial= 730
initial= str(initial)+' days'

#Period length that we perform the cross validation for.
period= 120
period=str(period)+' days'

#Horizon of prediction essentially for each fold.
horizon = 90
horizon=str(horizon)+' days'
fb_cv=cross_validation(m4,initial=initial,period=period,
horizon=horizon)

# Performance Metrics of fb_cv
cv = performance_metrics(fb_cv)
cv.mean()

In [0]:
f, ax = plt.subplots(figsize=(30, 5))

actual_index = within_covid.set_index(['ds'])
forecast_index = forecast4.set_index(['ds'])

actual_index['y'].plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)
forecast_index['yhat'].plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
plt.title('Actual v.s. Forecast')
plt.legend()
plt.show()

In [0]:
from prophet.plot import add_changepoints_to_plot
fig = m4.plot(forecast4)
a = add_changepoints_to_plot(fig.gca(), m4, forecast4)

In [0]:
from sklearn.metrics import mean_absolute_percentage_error,  r2_score, mean_squared_error

y_true = activations_index['y']
y_pred = forecast4['yhat'].iloc[:243]

print('mape:', mean_absolute_percentage_error(y_true, y_pred))
print('r2:', r2_score(y_true, y_pred))
print('RMSE', mean_squared_error(y_true, y_pred, squared=False))

## Adding Changing Points

In [0]:
m_changepoints = (
    pd.date_range('2019-03-01', '2019-06-30', periods=8).date.tolist() +
    pd.date_range('2020-01-01', '2020-12-31', periods=16).date.tolist() +
    pd.date_range('2021-01-01', '2021-12-31', periods=16).date.tolist() +
    pd.date_range('2022-01-01', '2022-06-01', periods=8).date.tolist()
)

In [0]:
m5 = Prophet(
            yearly_seasonality= False, 
            weekly_seasonality= True,
            changepoint_prior_scale= 0.5,
            seasonality_prior_scale=20,
            holidays_prior_scale=0.5,
            holidays= holidays,
            seasonality_mode = 'multiplicative'
                        )
m5.add_country_holidays(country_name='DE')
m5.add_seasonality('quarterly', period=91.25, fourier_order=5, mode='multiplicative')
#m4.add_seasonality('weekly', period=7, fourier_order=10, mode='multiplicative')

m5.add_seasonality(name='yearly_on_season', period=365, fourier_order=10, condition_name='on_season', mode='multiplicative')
m5.add_seasonality(name='yearly_off_season', period=365, fourier_order=10, condition_name='off_season', mode='multiplicative')

m5.add_regressor('cases', standardize='auto', mode='multiplicative')
#m4.add_regressor('deaths', standardize='auto', mode='multiplicative')

m5.fit(within_covid_activation)

#future4 = m4.make_future_dataframe(17, freq='W')
#future4['on_season'] = future4['ds'].apply(is_nfl_season)
#future4['off_season'] = ~future4['ds'].apply(is_nfl_season)

forecast5 = m5.predict(within_covid_activation)

fig = m5.plot_components(forecast5)

In [0]:
m5.plot(forecast5, uncertainty=True)
plt.show()

In [0]:
from prophet.plot import add_changepoints_to_plot
fig = m5.plot(forecast5)
a = add_changepoints_to_plot(fig.gca(), m5, forecast5)

In [0]:
# Initial training period.
initial= 730
initial= str(initial)+' days'

#Period length that we perform the cross validation for.
period= 120
period=str(period)+' days'

#Horizon of prediction essentially for each fold.
horizon = 90
horizon=str(horizon)+' days'
fb_cv=cross_validation(m5,initial=initial,period=period,
horizon=horizon)

# Performance Metrics of fb_cv
cv = performance_metrics(fb_cv)
cv.head()

cv.mean()

##with add seasonality is 0.15371

In [0]:
f, ax = plt.subplots(figsize=(30, 5))

actual_index = within_covid.set_index(['ds'])
forecast_index = forecast5.set_index(['ds'])

actual_index['y'].plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)
forecast_index['yhat'].plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
plt.title('Actual v.s. Forecast')
plt.legend()
plt.show()

In [0]:
from sklearn.metrics import mean_absolute_percentage_error,  r2_score, mean_squared_error

y_true = activations_index['y']
y_pred = forecast5['yhat'].iloc[:243]

print('mape:', mean_absolute_percentage_error(y_true, y_pred))
print('r2:', r2_score(y_true, y_pred))
print('RMSE', mean_squared_error(y_true, y_pred, squared=False))

#DO

In [0]:
future_real = pd.read_csv("/dbfs/FileStore/shared_uploads/chien.liu@hellofresh.com/W_input.csv", parse_dates=['ds'])
future_real = future_real[['ds', 'simulated']]
future_real.head()

In [0]:
future_real = future_real[future_real['ds'] > '2022-09-01']
# future_real_df = future_real_df[['ds', '']]
future_real_df_index = future_real.set_index(['ds'])
future_real_df_index

In [0]:
forecast_df = forecast5[['ds', 'yhat']]
forecast_df = forecast_df[forecast_df['ds'] > '2022-09-01']
forecast_df = forecast_df[forecast_df['ds'] < '2022-11-27'].set_index(['ds'])
forecast_df

In [0]:
f, ax = plt.subplots(figsize=(30, 5))

future_real_df_index.plot(kind='line',y='simulated', color='red', label='Actual', ax=ax)
forecast_df.plot(kind='line',y='yhat', color='green',label='Forecast', ax=ax)
plt.title('Prophet Actual v.s. Forecast')
plt.legend()
plt.show()

In [0]:
from sklearn.metrics import mean_absolute_percentage_error,  r2_score, mean_squared_error

y_true = future_real_df_index
y_pred = forecast_df

print('mape:', mean_absolute_percentage_error(y_true, y_pred))
print('r2:', r2_score(y_true, y_pred))
print('RMSE', mean_squared_error(y_true, y_pred, squared=False))